# Carbon Scripting

Showing snippets of Carbon scripting code from the 'Carbon Scripting' document.

In [1]:
#r "nuget: RCS.Carbon.Tables"
#r "nuget: RCS.Carbon.Import"
#r "nuget: RCS.Carbon.Export"

using System;
using System.IO;
using System.Text;
using System.Collections.Generic;
using System.Diagnostics;

using RCS.Carbon.Shared;
using RCS.Carbon.Tables;
using RCS.Carbon.Variables;
using RCS.Carbon.Import;
using RCS.Carbon.Export;

Installed Packages RCS.Carbon.Export, 8.5.8 RCS.Carbon.Import, 8.5.8 RCS.Carbon.Tables, 8.5.9

### Licensing
 
To use the Carbon library, you need a licence. Your normal Ruby licence will work, or you can 
use the standard Carbon Guest licence as in the examples here. Every script must execute at 
least the first two lines below

In [2]:
var eng = new CrossTabEngine();
var login = await eng.LoginId("16499372","C6H12O6");   // Carbon guest licence
Console.WriteLine($"User: {login.Name} DLL: {eng.Version}");

User: CarbonGuest DLL: 8.5.9.0


### The CrossTab Engine

CrossTabEngine is an object defined in RCS.Carbon.Tables.dll. You create one of these (here 
referenced as eng) and call its LoginId method, passing your user Id and password. The await
modifier is because the function is asynchronous and waits for the return value before 
proceeding. The licence call could fail for many reasons (internet down, wrong credentials, 
licence expired etc). If this happens the script aborts with an error message. The credentials 
here are for the Carbon guest licence which anyone can use.

Console.WriteLine shows how to give progress messages. Console.WriteLine can take a 
compound string. The leading $ means the text will be interspersed with objects – could be any 
of your variables or values. Anything inside {} is treated as an object reference and turned into 
a string. In this case the LicenceSummary object, referred to as login, has a member Name, 
referred to as login.Name, and the engine has a member called Version, referred to as 
eng.Version.

### Select a Job


The next step is to select a job. Local jobs are identified solely by their folder path. the folder C:\RedCentre is part of the Red Centre Software [Carbon Libraries and Applications](https://rcsapps.azurewebsites.net/doc/carbon/Introduction%20to%20the%20RedCentre%20Carbon%20Libraries%20and%20Applications.pdf).

In [3]:
var eng = new CrossTabEngine();
var login = await eng.LoginId("16499372","C6H12O6");   // guest licence

var jobdir = @"C:\RedCentre\Jobs\Demo";
eng.OpenJob(jobdir);
Console.WriteLine($"User: {login.Name} DLL: {eng.Version} Job: {jobdir}");

User: CarbonGuest DLL: 8.5.9.0 Job: C:\RedCentre\Jobs\Demo


### Startup.csx

In a command line environment, the below cell would be run as a seperate file. In a Notebook, we do it 'inline'.


In [4]:
// Declare and define global engine eng, dprops and sprops objects 
// Use eng to check the licence credentials
// Define SetJobDir wrapper
// Set some default table properties
var eng = new CrossTabEngine();
// Authentication
var summary = await eng.LoginId("16499372", "C6H12O6");
Console.WriteLine($"User: {summary.Name}");
Console.WriteLine($"DLL: {eng.Version}");
// Assign job
bool SetJobDir(string jobdir)
{
    try
    {
        eng.OpenJob(jobdir);
    }
    catch (Exception e)
    {
        Console.WriteLine($"Could not open job at {jobdir}. {e.Message}");
        return false;
    }
    Console.WriteLine($"Job: {eng.JobName}");
    return true;
}
// Default table properties for this job
var sprops = new XSpecProperties() {}; 
var dprops = new XDisplayProperties() {};
dprops.Output.Format = XOutputFormat.TSV;
dprops.Cells.Frequencies.Visible = true; 
dprops.Cells.ColumnPercents.Visible = true;
dprops.Cells.RowPercents.Visible = false;

User: CarbonGuest
DLL: 8.5.9.0


### OneTable.csx

In [5]:
eng.OpenJob(jobdir);
var ret = eng.GenTab("test", "gender(cwf;*)", "region(cwf;*)", "", "", sprops, dprops); // sprops and dprops are defined in startup.csx
Console.WriteLine(ret);

Name: test
Top: Respondent Gender
Side: region
	Cases WF	Male	Female
Cases WF	10000	4985	5015
NE	2522	1255	1267
		25.18%	25.26%
SE	2459	1213	1246
		24.33%	24.85%
SW	2475	1230	1245
		24.67%	24.83%
NW	2544	1287	1257
		25.82%	25.06%



### SaveTable.csx

In [6]:
if (!SetJobDir(@"C:\RedCentre\Jobs\Demo")) return;
var outfile = @"C:\RedCentre\SandPit\report.txt";
dprops.Cells.Frequencies.Visible = false; // change a display property
dprops.Output.Format = XOutputFormat.SSV; // space-aligned
var ret = eng.GenTab("test", "gender(cwf;*)", "region(cwf;*)", "", "", sprops, dprops);

File.AppendAllText(outfile, ret, Encoding.UTF8); // append to existing if present
Console.WriteLine($"Table saved to {outfile}");
Process.Start("Notepad", outfile);

Job: Demo
Table saved to C:\RedCentre\SandPit\report.txt


<null>

### Output Many Tables
You can run many tables and save to a single file using the internal OutputManager class. A 
typical sequence is:

In [7]:
eng.OutputManager.Open(XOutputFormat.TSV, @"docs\ManyTables.tsv");
eng.OutputManager.AppendLine("\n *** Gender by Region ***\n"); // \n is new line

eng.GenTab("tab1", "gender", "region", "", "", sprops, dprops);
eng.OutputManager.AppendTable();

eng.OutputManager.AppendLine("\n *** Age by Education ***\n");
eng.GenTab("tab2", "age", "edu", "", "", sprops, dprops);
eng.OutputManager.AppendTable();

eng.OutputManager.Close();
Console.WriteLine("Tables written to " + eng.OutputManager.Message); 

Tables written to C:\RedCentre\Jobs\Demo\Docs\docs\ManyTables.tsv


### DefCodeFrame

Use the DefCodeFrame object to create codeframes (*.MET files). This example creates the 
BrandX codes directly, the BrandY codes in a loop, and the BrandZ codes by adding multiple 
items in a single line. Nets for each brand are also added as a single line.

In [8]:
var jobdir = @"C:\RedCentre\Jobs\Demo";
if (!SetJobDir(jobdir)) return;
var dcf = eng.NewDefCodeFrame("Test1", "Example DefCodeFrame");
if (dcf==null)
{
 Console.WriteLine(eng.Message());
 return;
}
//BrandX
dcf.AddCode(1, "Brand1");
dcf.AddCode(2, "Brand2");
dcf.AddCode(3, "Brand3");
//BrandY
for(int i = 4; i<= 6; i++) 
dcf.AddCode(i, $"Brand{i}");
//BrandZ
dcf.AddItems("7=Brand7\n8=Brand8\n9=Brand9\n10=Brand10=BrandY");
//Nets
dcf.AddItems("_net(1/3)=BrandX\n_net(4/6)=BrandZ\n_net(7/10)=BrandZ");
if (!dcf.Close()) Console.WriteLine(eng.Message());
else Console.WriteLine(@$"Test1 saved in {jobdir}\CaseData");

Job: Demo
Test1 saved in C:\RedCentre\Jobs\Demo\CaseData


<null>

### DefCon

You create a DefCon object, here referenced as dcn, passing the variable name and description, and then call AddItem() and other routines to build the construction.

This just writes the MET file, which you could construct with eng.Construct(varname) or by passing true in the Close call `dcn.Close(true)`;

The DefCon subroutines are AddItem(), AddInc(), AddNextItem(), AddExpr(), SetFlags() and Close(). See Appendix 4 below for details.

In [9]:
var jobdir = @"C:\RedCentre\Jobs\Demo";
if (!SetJobDir(jobdir)) return;
var dcn = eng.NewDefCon("Test2", "Example DefCon");
if (dcn==null)
{
 Console.WriteLine(eng.Message());
 return;
}
dcn.AddItem(1, "Age(1/2)", "15yrs to 35yrs");
dcn.AddItem(2, "Age(3/4)", "36yrs to 65yrs");
dcn.AddItem(3, "Age(5)", "65yrs and above");
dcn.Close(true);
Console.WriteLine(@$"Test2 saved in {jobdir}\CaseData");

Job: Demo
Test2 saved in C:\RedCentre\Jobs\Demo\CaseData


<null>

### DefGen

Use the DefGen object to generate the metadata (*.met) specification for constructed variables. First, we create a codeframe called AidedBrandList, then the generator uses the new code frame
as the outer level (or top axis) of a Net Promoter Score (NPS) ratings grid

In [10]:
var jobdir = @"C:\RedCentre\Jobs\Demo";
if (!SetJobDir(jobdir)) return;
var dcf = eng.NewDefCodeFrame("AidedBrandList", "");
if (dcf==null)
{
 Console.WriteLine(eng.Message());
 return;
}
dcf.AddCode(1, "BrandX");
dcf.AddCode(2, "BrandY");
dcf.AddCode(3, "BrandZ");
dcf.Close();
var dgn = eng.NewDefGen("Test3", "NPS_$a($b)", "", "Example DefGen");
if (dgn==null)
{
 Console.WriteLine(eng.Message());
 return;
}
dgn.AddLevel("a", "Brand", "AidedBrandList");
dgn.AddLevel("b", "Rating", "NPS_1");
dgn.Close(true);
Console.WriteLine(@$"AidedBrandList and Test3 saved in {jobdir}\CaseData");

Job: Demo
AidedBrandList and Test3 saved in C:\RedCentre\Jobs\Demo\CaseData


<null>

### DefWght

Use the DefWght object to create weight variables.

In [11]:
var jobdir = @"C:\RedCentre\Jobs\Demo";
if (!SetJobDir(jobdir)) return;
var dwt = eng.NewDefWght("Test4", "Cell", "Month", "", 0, 0, "Example DefWght");
if (dwt==null)
{
 Console.WriteLine(eng.Message());
 return;
}
dwt.AddTarget("Age", "10;30;30;20;10");
dwt.AddTarget("Gender", "40;60");
dwt.AddTarget("Region", "25;25;25;25");
dwt.Close(true);
Console.WriteLine(@$"Test4 saved in {jobdir}\CaseData");

Job: Demo
Test4 saved in C:\RedCentre\Jobs\Demo\CaseData


<null>

### DefGrid

Use the DefGrid object to create grid variables. This is an alternative to using the more general DefGen and does not require intermediate codeframes.

In [12]:
var jobdir = @"C:\RedCentre\Jobs\Demo";
if (!SetJobDir(jobdir)) return;
var dgr = eng.NewDefGrid("Test5","copy of NPS", "Brand", "NPS_1;NPS_2;NPS_3", "Score", 
"NPS_1");
if (dgr==null)
{
 Console.WriteLine(eng.Message());
 return;
}
dgr.Close(true);
Console.WriteLine(@$"Test5 saved in {jobdir}\CaseData");

Job: Demo
Test5 saved in C:\RedCentre\Jobs\Demo\CaseData


<null>

### DefNet

Use the DefNet object to create an arbitrary set of nets at any hierarchic level.

In [13]:
var jobdir = @"C:\RedCentre\Jobs\Demo";
if (!SetJobDir(jobdir)) return;
var dn = eng.NewDefNet("Test6", "Example DefNet");
 var dnl = dn.AddLevel("Region");
 dnl.AddItem("region(1/2)", "North");
 dnl.AddItem("region(3/4)", "South");
 dnl = dn.AddLevel("Work");
 dnl.AddItem("occupation(1/2)", "Professional");
 dnl.AddItem("occupation(2/3)", "Midlevel");
 dnl.AddItem("occupation(4/5)", "Other");
dn.Close(true);
Console.WriteLine(@$"Test6 saved in {jobdir}\CaseData");

Job: Demo
Test6 saved in C:\RedCentre\Jobs\Demo\CaseData


<null>

### DefDates

Use the DefDates object to create date variables as Modified Julian Dates (the day number since 17Nov1858)

In [14]:
var jobdir = @"C:\RedCentre\Jobs\Demo";
if (!SetJobDir(jobdir)) return;
var ddt = eng.NewDefDates("Test7", "Date", "1y 1Jan2001", "Day", "mmmyyyy", "Example DefDates");
if (ddt==null)
{
 Console.WriteLine(eng.Message());
 return;
}
ddt.Close(true);
Console.WriteLine(@$"Test7 saved in {jobdir}\CaseData");

Job: Demo
Test7 saved in C:\RedCentre\Jobs\Demo\CaseData


<null>

### Title Text Modes

The Top and Side title text can be auto-generated according to four display property settings.

In [15]:
if (!SetJobDir(@"C:\RedCentre\Jobs\Demo")) return;
dprops.Titles.Labelling.Script = true;
dprops.Titles.Labelling.Codes = true;
dprops.Titles.Labelling.Name = true;
dprops.Titles.Labelling.Desc = true;

var ret = eng.GenTab("test", "EDU(1/3)", "Gender(cwf;*)", "", "",sprops,dprops);
Console.WriteLine(ret);

Job: Demo
Name: test
Top: EDU(1/3)
Side: Gender(cwf;*)
          +------+------+------+
          |LT HIG|HIGH S|JUNIOR|
          |H SCHO|CHOOL | COLLE|
          |OL    |      |GE    |
----------+------+------+------+
Cases WF  |1448  |5353  |736   |
----------+------+------+------+
Male      |50.07%|50.31%|47.69%|
----------+------+------+------+
Female    |49.93%|49.69%|52.31%|
----------+------+------+------+



<null>

### ImportDemoDems.csx

In [16]:
var jobdir = @"C:\RedCentre\Sandpit\TestImport";

if (!SetJobDir(jobdir)) return;
var sourcefile = "Demo_dems_2021.sav"; // relative to <jobdir>\Source\
var imp = new ImportEngine();
imp.AttachJob(eng);
ImportSavSettings settings = new ImportSavSettings();
settings.TryBlend = false;
Console.WriteLine("Importing: " + sourcefile);
var ret = imp.ImportSAV(sourcefile, settings);
Console.WriteLine(ret);

Job: TestImport
Importing: Demo_dems_2021.sav


Error: System.IO.FileNotFoundException: Could not find file 'C:\RedCentre\Sandpit\TestImport\Source\Demo_dems_2021.sav'.
File name: 'C:\RedCentre\Sandpit\TestImport\Source\Demo_dems_2021.sav'
   at Microsoft.Win32.SafeHandles.SafeFileHandle.CreateFile(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.FileStreamHelpers.ChooseStrategyCore(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileA

### ExportVariables.csx

In [17]:
var jobdir = @"C:\RedCentre\Sandpit\TestExport";
if (!SetJobDir(jobdir)) return;
string exportvars = "Region,AgeX,UBA";
var exp = new ExportEngine();
exp.AttachJob(eng);
var settings = new ExportSettings("TestOut")
{
Format = VExportType.Tableau
};
var result = exp.ExportVars(exportvars, settings);
Console.WriteLine(result);

Job: TestExport
Wrote three files with stem TestOut to C:\RedCentre\Sandpit\TestExport


<null>

# Accessing Cloud Jobs

The only difference for accessing a cloud job is the OpenJob command. If called with two parameters, as customer account and job name, then the Azure connection is automatically 
established, and all case data reads will be from the Azure storage. A stable and fast internet connection is advised.

In [18]:
var jobname = "gss"; // USA General Social Survey
var eng = new CrossTabEngine(); 
await eng.LoginId("16499372","C6H12O6"); // guest licence
eng.OpenJob("rcspublic", jobname); // Azure access happens here
Console.WriteLine($"Azure job opened: {jobname}");
var dprops = new XDisplayProperties();
dprops.Output.Format = XOutputFormat.SSV; // space formatted
var sprops = new XSpecProperties();
var ret = eng.GenTab("Tab1", "Race(cwf%;*)", "Degree(cwf%;*)", "", "", sprops, dprops);
Console.WriteLine(ret);

Azure job opened: gss
Name: Tab1
Top: Race of respondent
Side: r's highest degree
          +------+------+------+------+------+
          |Cases |IAP   |WHITE |BLACK |OTHER |
          |WF    |      |      |      |      |
----------+------+------+------+------+------+
Cases WF  |68846 |0     |52033 |9187  |3594  |
----------+------+------+------+------+------+
less than |13833 |0     |9920  |2696  |971   |
----------+------+------+------+------+------+
HIGH SCHOO|34792 |0     |26967 |4721  |1507  |
----------+------+------+------+------+------+
associate/|4038  |0     |2876  |564   |228   |
----------+------+------+------+------+------+
bachelor's|10511 |0     |8127  |801   |547   |
----------+------+------+------+------+------+
GRADUATE  |5476  |0     |4031  |353   |332   |
----------+------+------+------+------+------+
IAP       |0     |0     |0     |0     |0     |
----------+------+------+------+------+------+
DK        |30    |0     |21    |7     |2     |
----------+------+------+

### ExportVars_Azure.csx

This example shows how to send outputs to a local directory.

In [19]:
var jobname = "gss"; // USA General Social Survey
var eng = new CrossTabEngine(); 
await eng.LoginId("16499372","C6H12O6"); // guest licence
eng.OpenJob("rcspublic", jobname); // Azure access happens here
Console.WriteLine($"Azure job opened: {jobname}");
eng.SetHomeDir(@"C:\RedCentre\Sandpit"); // Azure data will arrive here
string exportvars = "race,Age8,sex";
var exp = new ExportEngine();
exp.AttachJob(eng);
var settings = new ExportSettings("TestOut")
{
Format = VExportType.Tableau
};
var result = exp.ExportVars(exportvars, settings);
Console.WriteLine(result);

Azure job opened: gss
Wrote three files with stem TestOut to C:\RedCentre\Sandpit
